# Kamus Kata

In [72]:
import pandas as pd
import json
import re
import numpy as np
import seaborn as sns
import math
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

def display_options():
     
    display = pd.options.display
    display.max_columns = 5
    display.max_rows = 100
    display.max_colwidth = 222
    display.width = None
    return None
 
display_options()

memuat kata dasar

In [73]:
file = open("./kata-dasar.json")
words = json.load(file)

# Label with Parent

## Parent Label with Transaction Type

query_labels_parent_with_type = '''SELECT a.id, CASE WHEN a.nama_transaksi IS NULL THEN 'Tanpa Label' ELSE a.nama_transaksi END as nama_transaksi, CASE WHEN c.jenis = 'debit' THEN 0 ELSE 1 END as jenis, COUNT(c.id) as transaction_count,
b.saldo_kas_id 
FROM (
    SELECT aa.nama_transaksi, aa.id, ab.jenis, aa._lft, aa._rgt FROM jenis_transaksis AS aa
        LEFT JOIN transaksi AS ab ON ab.jenis_transaksi_id = aa.id
        WHERE aa.parent_id IS NULL
        GROUP BY aa.id, aa.nama_transaksi, aa._lft, aa._rgt, ab.jenis
) AS a 
    RIGHT JOIN jenis_transaksis AS b ON b._lft BETWEEN a._lft AND a._rgt 
    RIGHT JOIN transaksi AS c ON b.id = c.jenis_transaksi_id 
GROUP BY a.id, a.nama_transaksi, c.jenis, b.saldo_kas_id'''

parents_label = pd.read_sql(query_labels_parent_with_type, mydb)
parents_label.to_json("dataset.json")

In [74]:
file = open('./dataset.json')
parents_label = pd.read_json(file)

## Fungsi LowerCase

In [75]:

def lowerArray(arr):
	result = list()
	for word in arr:
		result.append(word.lower())
	return result

#### Testing LowerCase

In [76]:
test_words = ["Hello", "World"]
if lowerArray(test_words)[0] == 'hello':
	print("✅")
else:
	print("🔥")

✅


## Fungsi Filter Char

In [77]:
def filterCharArray(arr):
	result = list()
	for word in arr:
		string_text_only = re.sub(r"[^a-zA-Z\s]", " ", word)
		string_space_trimmed = re.sub(r"\s+", " ", string_text_only)
		result.append(string_space_trimmed.strip())

	return result

#### Testing FilterChar

In [78]:
test_words = ["persembahan rutin kebaktian utama ii :", "wor|d "]
print(filterCharArray(test_words))
if filterCharArray(test_words)[1] == 'word':
	print("✅")
else:
	print("🔥")

['persembahan rutin kebaktian utama ii', 'wor d']
🔥


### Fungsi Stemming

In [79]:
def stemmingArray(arr):
	result = list()
	for data in arr:
		result_words = list()
		for word in data:
			if word in words:
				result_words.append(word)
			else:
				word_temp = word
				# se
				word_temp = re.sub(r"^se", "", word_temp)
				if word_temp in words:
					result_words.append(word_temp)
					continue
				
				# di
				word_temp = re.sub(r"^di", "", word_temp)
				if word_temp in words:
					result_words.append(word_temp)
					continue
				# mem-b
				word_temp = re.sub(r"^mem([bp])(.*)$", r"\1\2", word_temp)
				if word_temp in words:
					result_words.append(word_temp)
					continue
				
				# hilangkan imbuhan meng?
				word_temp = re.sub(r"^meng([aiueoghk].*)$", r"\1", word_temp)
				if word_temp in words:
					result_words.append(word_temp)
					continue
				
				# hilangkan imbuhan men[c|d]
				word_temp = re.sub(r"^men([cdjt].*)$", r"\1", word_temp)
				if word_temp in words:
					result_words.append(word_temp)
					continue
				
				# hilangkan imbuhan per?

				word_temp = re.sub(r"^per([hgk].*)$", r"\1", word_temp)
				if word_temp in words:
					result_words.append(word_temp)
					continue

				# hilangkan imbuhan pen

				word_temp = re.sub(r"^pen([jdcz].*)$", r"\1", word_temp)
				if word_temp in words:
					result_words.append(word_temp)
					continue


				# hilangkan imbuhan pem

				word_temp = re.sub(r"^pem([bfv].*)$", r"\1", word_temp)
				if word_temp in words:
					result_words.append(word_temp)
					continue


				# hilangkan imbuhan ke

				word_temp = re.sub(r"^ke", "", word_temp)
				if word_temp in words:
					result_words.append(word_temp)
					continue

				
				# hilangkan imbuhan meny -> mens
				word_temp = re.sub(r"^meny(.*)$", r"s\1", word_temp)
				if word_temp in words:
					result_words.append(word_temp)
					continue
				
				
				# hilangkan imbuhan me[lmnrw]
				word_temp = re.sub(r"^me([lmnrw].*)$", r"\1", word_temp)
				if word_temp in words:
					result_words.append(word_temp)
					continue
				
				# hilangkan imbuhan peny -> pes

				word_temp = re.sub(r"^peny(.*)$", r"s\1", word_temp)
				if word_temp in words:
					result_words.append(word_temp)
					continue
				
				# hilangkan imbuhan pen -> pet

				word_temp = re.sub(r"^pen(.*)$", r"t\1", word_temp)
				if word_temp in words:
					result_words.append(word_temp)
					continue
				
				# hilangkan imbuhan pem -> pep

				word_temp = re.sub(r"^pem(.*)$", r"p\1", word_temp)
				if word_temp in words:
					result_words.append(word_temp)
					continue
				
				# hilangkan imbuhan pe

				word_temp = re.sub(r"^pe([lmnrwy].*)$", r"\1", word_temp)
				if word_temp in words:
					result_words.append(word_temp)
					continue
				
				# hilangkan akhiran k?an

				word_temp = re.sub(r"(ku|nya|mu|kah|lah|pun)$", "", word_temp)
				if word_temp in words:
					result_words.append(word_temp)
					continue

				word_temp = re.sub(r"(i|an)$", "", word_temp)
				if word_temp in words:
					result_words.append(word_temp)
					continue

				word_temp = re.sub(r"(k)$", "", word_temp)
				if word_temp in words:
					result_words.append(word_temp)
					continue

				result_words.append(word)
		result.append(result_words)
	return result

#### Testing Stemming

In [80]:
test_words = [["pelunasan", "dimakan", "agus"]]

print(stemmingArray(test_words))

if stemmingArray(test_words)[0][0] == "lunas":
	print("✅")
else:
	print("🔥")

[['lunas', 'makan', 'agus']]
✅


## Fungsi BoW (Bag Of Words)

In [81]:
def getBowArray(arr):
	result_wordcount = dict()
	wordcount = dict()
	wordcountinlabelcollection = dict()
	result_dataset = list()
	for index, data in enumerate(arr):

		wordlabelrecord = list()

		for word in data:
			
			# jika kata belum a da di collection

			if word not in result_dataset:
				wordcountinlabelcollection[word] = 1
				wordlabelrecord.append(word)
				result_dataset.append(word)
				wordcount[word] = 1
			else:
				# jika sudah ada tapi belum ada di label record
				# maka tambah
				if word not in wordlabelrecord:
					wordcountinlabelcollection[word] += 1
					wordlabelrecord.append(word)
				wordcount[word] += 1
		
	result_wordcount['data'] = result_dataset
	result_wordcount['count'] = wordcount
	result_wordcount['label_count'] = wordcountinlabelcollection
	return result_wordcount

#### Testing BoW

In [82]:
test_words = [["saya", "makan", "makanan", "saya"]]

print(getBowArray(test_words))

if getBowArray(test_words)['count']['saya'] == 2:
	print("✅")
else:
	print("🔥")

{'data': ['saya', 'makan', 'makanan'], 'count': {'saya': 2, 'makan': 1, 'makanan': 1}, 'label_count': {'saya': 1, 'makan': 1, 'makanan': 1}}
✅


### Fungsi Split Word

In [83]:
def splitwordArray(datasets):
	result = list()
	for word in datasets:
		result.append(word.split(" "))
	return result

In [84]:
test_words = ["apa kabar", "halo dunia"]
if splitwordArray(test_words)[0][0] == "apa":
	print("✅")
else:
	print("🔥")

✅


## Fungsi StopWord

In [85]:
file_stopwords = open("./stopwords/id.json")
stopwords = json.load(file_stopwords)

def remove_stopwords(arr):
	result_dataset = list()
	for words in arr:
		result_words = list()
		for word in words:
			if word not in stopwords:
				result_words.append(word)
		result_dataset.append(result_words)
	return result_dataset

#### Test Stopword

In [86]:
test_words = [["berburu", "dan", "meramu"]]
if remove_stopwords(test_words)[0] == ['berburu', "meramu"]:
	print("✅")
else:
	print("🔥")

✅


# Prepocessing

## Casefolding

In [87]:
print(parents_label['nama_transaksi'])
datasets_lowered = np.array(lowerArray(parents_label['nama_transaksi']))
datasets_filtered = filterCharArray(datasets_lowered)
print(datasets_lowered)

0                              Pencatatan Saldo Tunai Kas Rutin
1                               Pencatatan Saldo Bank Kas Rutin
2                         Pencatatan Saldo Bank Kas Pembangunan
3                            Pencatatan Saldo Bank Kas Cadangan
4                        Pencatatan Saldo Tunai Kas Pembangunan
                                 ...                           
583                                   Tinta Epson T 664 (Hitam)
584                Biaya Konsumsi Sekretariat dan Aguamor 2 Dos
585    Biaya Foto Copy Liturgi dan Warta Jemaat (05 Maret 2023)
586                             Pencatatan Saldo Bank Kas Rutin
587                         Persembahan Rutin Kebaktian Utama I
Name: nama_transaksi, Length: 588, dtype: object
['pencatatan saldo tunai kas rutin' 'pencatatan saldo bank kas rutin'
 'pencatatan saldo bank kas pembangunan'
 'pencatatan saldo bank kas cadangan'
 'pencatatan saldo tunai kas pembangunan'
 'persembahan rutin kebaktian utama ii :'
 'kolekte tang

## Tokenization

In [88]:
datasets_tokenized = splitwordArray(datasets_filtered)
print(datasets_tokenized)
tokenization_datasets = pd.DataFrame(datasets_tokenized)

[['pencatatan', 'saldo', 'tunai', 'kas', 'rutin'], ['pencatatan', 'saldo', 'bank', 'kas', 'rutin'], ['pencatatan', 'saldo', 'bank', 'kas', 'pembangunan'], ['pencatatan', 'saldo', 'bank', 'kas', 'cadangan'], ['pencatatan', 'saldo', 'tunai', 'kas', 'pembangunan'], ['persembahan', 'rutin', 'kebaktian', 'utama', 'ii'], ['kolekte', 'tanggu', 'merah', 'kebaktian', 'utama', 'i'], ['persembahan', 'rutin', 'kebaktian', 'utama', 'iii'], ['kolekte', 'tanggu', 'merah', 'kebaktian', 'utama', 'iii'], ['kolekte', 'tanggu', 'merah', 'kebaktian', 'utama', 'ii'], ['persembahan', 'rutin', 'kebaktian', 'utama', 'i'], ['penerimaan', 'rayon'], ['tinta', 'epson', 't', 'hitam'], ['biaya', 'konsumsi', 'sekretariat', 'dan', 'aguamor', 'dos'], ['biaya', 'foto', 'copy', 'liturgi', 'dan', 'warta', 'jemaat', 'maret'], ['biaya', 'snack', 'rapat', 'anggota', 'seksi', 'acara', 'panitia', 'paskah', 'bahari', 'klasis', 'kota', 'kupang', 'timur', 'senin', 'maret'], ['biaya', 'pertamax', 'untuk', 'genset', 'gereja'], ['bi

## Stopword

In [89]:
datasets_stopwords_remove = remove_stopwords(datasets_tokenized)

datasets_stopwords = pd.DataFrame(datasets_stopwords_remove)
print(datasets_stopwords)

              0         1   ...    13    14
0     pencatatan     saldo  ...  None  None
1     pencatatan     saldo  ...  None  None
2     pencatatan     saldo  ...  None  None
3     pencatatan     saldo  ...  None  None
4     pencatatan     saldo  ...  None  None
..           ...       ...  ...   ...   ...
583        tinta     epson  ...  None  None
584        biaya  konsumsi  ...  None  None
585        biaya      foto  ...  None  None
586   pencatatan     saldo  ...  None  None
587  persembahan     rutin  ...  None  None

[588 rows x 15 columns]


## Stemming

In [90]:
datasets_stemming = stemmingArray(datasets_stopwords_remove)
print(datasets_stemming)

datasets_stemming_pd = pd.DataFrame(datasets_stemming)

[['catat', 'saldo', 'tunai', 'kas', 'rutin'], ['catat', 'saldo', 'bank', 'kas', 'rutin'], ['catat', 'saldo', 'bank', 'kas', 'bangun'], ['catat', 'saldo', 'bank', 'kas', 'cadang'], ['catat', 'saldo', 'tunai', 'kas', 'bangun'], ['persembahan', 'rutin', 'bakti', 'utama', 'ii'], ['kolekte', 'tanggu', 'merah', 'bakti', 'utama', ''], ['persembahan', 'rutin', 'bakti', 'utama', 'iii'], ['kolekte', 'tanggu', 'merah', 'bakti', 'utama', 'iii'], ['kolekte', 'tanggu', 'merah', 'bakti', 'utama', 'ii'], ['persembahan', 'rutin', 'bakti', 'utama', ''], ['terima', 'rayon'], ['tinta', 'epson', 't', 'hitam'], ['biaya', 'konsumsi', 'sekretariat', 'aguamor', 'dos'], ['biaya', 'foto', 'copy', 'liturgi', 'warta', 'jemaat', 'maret'], ['biaya', 'snack', 'rapat', 'anggota', 'seksi', 'acara', 'panitia', 'paskah', 'bahari', 'klasis', 'kota', 'kupang', 'timur', 'senin', 'maret'], ['biaya', 'pertamax', 'untuk', 'genset', 'gereja'], ['biaya', 'fotocopy', 'liturgi', 'warta', 'lapor', 'uang'], ['kolekte', 'ibadah', 'pa

## Bag Of Words

In [91]:
bag_of_words = getBowArray(datasets_stemming)

bow_words_list = bag_of_words['count'].keys()

bag_of_words_pd = pd.DataFrame(bow_words_list)
print(bag_of_words_pd)


              0
0         catat
1         saldo
2         tunai
3           kas
4         rutin
..          ...
372      rohani
373  cendramata
374     singers
375    operator
376         lcd

[377 rows x 1 columns]


### Termweight

In [92]:
def get_dataset_wordcount(word: str, bow: dict) -> int:
	if word in bow['count']:
		return bow['count'][word] or 0
	else:
		return 0
	
def get_word_in_labels(word: str, bow_labels: dict) -> int:
	if word in bow_labels:
		return bow_labels[word] or 0
	else:
		return 0;
	
class TFResult:
	def __init__(self, label: list, labely: list, labelIndexInBow: list, labelIndexInLabelY: list, labelIndexInLabels: list):
		self.label = label
		self.labely = labely
		self.inBow = labelIndexInBow
		self.inLabelY = labelIndexInLabelY
		self.inLabels = labelIndexInLabels
		
		label_word_count = len(label)
		label_word_in_label_y = 0
		label_word_in_label_collection = 0

		for count in labelIndexInLabelY:
			label_word_in_label_y = label_word_in_label_y + count

		self.percentageInLabelY = 0

		if label_word_in_label_y != 0 and label_word_count != 0:
			self.percentageInLabelY = label_word_in_label_y / label_word_count
		
		self.weights = list()

		samples_count = len(datasets_stemming)

		weight_total = 0.0

		for index, word in enumerate(label):
			
			word_count_in_document = self.inBow[index]
			word_in_samples_count = self.inLabels[index]

			nPDF = 0

			if samples_count != 0 :
				nPDF = samples_count / (word_in_samples_count + 1)
			
			logNDF = math.log(nPDF)

			weight = 0

			if nPDF == 0:
				weight = 0
			else:
				weight = word_count_in_document * logNDF

			weight_total = weight_total + weight

			self.weights.append(weight)

		self.weight = weight_total / len(self.weights)


	
def getTF(label, label_y, bow: dict):

	# koleksi dari kata dari label terhadap label y
 
	labelIndexInLabelY = list()
	labelIndexInLabels = list()
	labelIndexInBow = list()

	for word in label:
		# menghitung kata yang pada label terhadap label_y
		
		word_index_to_y_count = 0
		word_index_to_labels_count = get_word_in_labels(word=word, bow_labels=bow['label_count'])
		word_index_to_bow = get_dataset_wordcount(word, bow)

		for wordy in label_y:
			if word == wordy:
				word_index_to_y_count = word_index_to_y_count + 1
		
		labelIndexInLabelY.append(word_index_to_y_count)
		labelIndexInBow.append(word_index_to_bow)
		labelIndexInLabels.append(word_index_to_labels_count)
	
	result = TFResult(label=label, labely=label_y, labelIndexInBow=labelIndexInBow, labelIndexInLabelY=labelIndexInLabelY, labelIndexInLabels=labelIndexInLabels) 

	return result

Rumus Termweight:

w<sub>ij</sub> = tf<sub>ij</sub> . log(N/df)

weight dari kata input terhadap kata sampel sama dengan 
1. *tfij* jumlah kata yang muncul pada sampel 
2. dikalikan log(jumlah koleksi sampel / jumlah kata dalam input muncul dalam koleksi sampel)

In [94]:
# simpan data weight di sini

datasets_w_i_by_j = list()

max_word_count = 0

for key in bag_of_words['count'].keys():
	if bag_of_words['count'][key] >= max_word_count:
		max_word_count = bag_of_words['count'][key];

for index_label, label in enumerate(datasets_stemming):
	
	datasets_w_i_by_j_label = list()

	for index_y_label, y_label in enumerate(datasets_stemming):
		data_tf = getTF(label=label, label_y=y_label, bow=bag_of_words)
		datasets_w_i_by_j_label.append(data_tf.weight)

	# menambahkan input terhadap jenis transaksi
  
	transaction_type = parents_label['jenis'][index_label]
	
	datasets_w_i_by_j_label.append(parents_label['saldo_kas_id'][index_label]*max_word_count)

	if transaction_type == 0:
		transaction_type = ((max_word_count * math.log(1/(1+1))) * 5) * -1
	else:
		transaction_type = 0

	datasets_w_i_by_j_label.append(transaction_type)

	datasets_w_i_by_j.append(datasets_w_i_by_j_label)

df = pd.DataFrame(datasets_w_i_by_j)

kmeans = KMeans(n_clusters=6, random_state=0).fit(df)
# print(kmeans.labels_)

result_labels = list()

for i, title in enumerate(parents_label['nama_transaksi']):
	result_labels.append([
		kmeans.labels_[i], title, parents_label['jenis'][i], parents_label['saldo_kas_id'], parents_label['transaction_count'][i],
	])

df = pd.DataFrame(result_labels, columns=('kelompok', 'label', 'jenis (0=debit)', 'saldo kas', 'total_transaksi'))
display(df)

<bound method DataFrame.to_markdown of      kelompok                                                     label  \
0           2                          Pencatatan Saldo Tunai Kas Rutin   
1           2                           Pencatatan Saldo Bank Kas Rutin   
2           2                     Pencatatan Saldo Bank Kas Pembangunan   
3           2                        Pencatatan Saldo Bank Kas Cadangan   
4           2                    Pencatatan Saldo Tunai Kas Pembangunan   
..        ...                                                       ...   
583         3                                 Tinta Epson T 664 (Hitam)   
584         3              Biaya Konsumsi Sekretariat dan Aguamor 2 Dos   
585         3  Biaya Foto Copy Liturgi dan Warta Jemaat (05 Maret 2023)   
586         4                           Pencatatan Saldo Bank Kas Rutin   
587         4                       Persembahan Rutin Kebaktian Utama I   

     jenis (0=debit)  \
0                  0   
1           

Visualisasi